In [1]:
import pandas as pd
import random
from collections import defaultdict

url = "https://docs.google.com/spreadsheets/d/1tJP_mT90nayY-MNB-_HsY2s-Phv8yP2OAZOfnvu8jhs/gviz/tq?tqx=out:csv"

champions_data = pd.read_csv(url)

In [2]:


# Function to get feedback based on comparison between guessed and target champion
def get_feedback(guess, target_data, champions_data):
    guess_data = champions_data[champions_data["Champion"] == guess].iloc[0]
    target_data = champions_data[champions_data["Champion"] == target_data].iloc[0]
    
    feedback = {}

    # Compare each attribute and store feedback
    for attribute in ["Gender", "Position(s)", "Species", "Resource", "Range type", "Region(s)"]:
        guess_value = guess_data[attribute]
        target_value = target_data[attribute]
        
        if guess_value == target_value:
            feedback[attribute] = {"type": "exact", "value": guess_value}
        elif isinstance(target_value, list) and any(item in target_value for item in guess_value):
            feedback[attribute] = {"type": "partial", "value": guess_value}
        else:
            feedback[attribute] = {"type": "mismatch", "value": guess_value}

    # Handle Release year with specific high/low feedback
    guess_year = guess_data["Release year"]
    target_year = target_data["Release year"]
    if guess_year == target_year:
        feedback["Release year"] = {"type": "exact", "value": guess_year}
    elif guess_year < target_year:
        feedback["Release year"] = {"type": "higher", "value": guess_year}
    else:
        feedback["Release year"] = {"type": "lower", "value": guess_year}
    
    return feedback

# Filtering functions based on feedback types
def filter_by_exact(data, attribute, value):
    return data[data[attribute] == value]

def filter_by_partial(data, attribute, value):
    return data[data[attribute].apply(lambda x: any(item in x for item in value))]

def filter_by_mismatch(data, attribute, value):
    return data[data[attribute] != value]

def filter_by_year(data, year, direction):
    if direction == "higher":
        return data[data["Release year"] > year]
    elif direction == "lower":
        return data[data["Release year"] < year]
    else:
        return data[data["Release year"] == year]

# Refine guesses based on feedback
def refine_guess(data, feedback):
    for attribute, details in feedback.items():
        if details["type"] == "exact":
            data = filter_by_exact(data, attribute, details["value"])
        elif details["type"] == "partial":
            data = filter_by_partial(data, attribute, details["value"])
        elif details["type"] == "mismatch":
            data = filter_by_mismatch(data, attribute, details["value"])
        elif attribute == "Release year":
            # Check if direction is available in feedback
            if details["type"] in ["higher", "lower"]:
                data = filter_by_year(data, details["value"], details["type"])
            else:  # If the year is an exact match
                data = filter_by_year(data, details["value"], "exact")
    return data


# Score each possible guess based on minimizing remaining champions
def score_guess(guess, remaining_champions, champions_data):
    feedback_groups = defaultdict(list)
    
    for candidate in remaining_champions["Champion"]:
        if candidate != guess:
            feedback = get_feedback(guess, candidate, champions_data)
            feedback_str = str(feedback)  # Convert feedback to a string to use as a unique key
            feedback_groups[feedback_str].append(candidate)

    # Calculate the average size of remaining groups after this guess
    avg_remaining_size = sum(len(group) for group in feedback_groups.values()) / len(feedback_groups)
    return avg_remaining_size

# Select the best guess to minimize remaining options
def select_optimal_guess(remaining_champions, champions_data):
    best_guess = None
    best_score = float('inf')

    for guess in remaining_champions["Champion"]:
        score = score_guess(guess, remaining_champions, champions_data)
        if score < best_score:
            best_score = score
            best_guess = guess
    print(f"guessing: {best_guess}")
    return best_guess

# Main simulation function with tie-breaking for similar champions
def simulate_guesses(target_champion, initial_champion, champions_data):
    remaining_champions = champions_data.copy()
    guess = initial_champion
    guesses = 1
    last_year_feedback = None

    while True:
        guesses += 1
        feedback = get_feedback(guess, target_champion, champions_data)
        remaining_champions = refine_guess(remaining_champions, feedback)
        
        # Check if we’ve narrowed it down to the target champion
        if len(remaining_champions) == 1 and remaining_champions.iloc[0]["Champion"] == target_champion:
            return guesses
        # Edge case: If two champions remain with identical attributes except for Release year
        if len(remaining_champions) == 2:
            remaining_years = remaining_champions["Release year"].values
            if all(row.drop(["Champion", "Release year"]).equals(remaining_champions.iloc[0].drop(["Champion", "Release year"])) 
                   for _, row in remaining_champions.iterrows()):
                
                # Use Release year feedback to select the correct champion
                if last_year_feedback == "higher":
                    guess = remaining_champions[remaining_champions["Release year"] == min(remaining_years)]["Champion"].values[0]
                elif last_year_feedback == "lower":
                    guess = remaining_champions[remaining_champions["Release year"] == max(remaining_years)]["Champion"].values[0]
                else:
                    # If no feedback on Release year yet, guess the champion with the median year
                    guess = remaining_champions.iloc[(remaining_champions["Release year"] - remaining_champions["Release year"].median()).abs().argsort()[:1]]["Champion"].values[0]
                continue  # Skip rest of the loop and guess again with updated logic
        
        # Use the optimal guessing strategy if no special tiebreaker is required
        guess = select_optimal_guess(remaining_champions, champions_data)
        
        # Update last year feedback based on the current feedback
        if "Release year" in feedback:
            last_year_feedback = feedback["Release year"]["type"]

def find_max_guesses(initial_champion, champions_data):
    max_guesses = 0
    champion_guesses = {}

    for target_champion in champions_data["Champion"]:
        if target_champion != initial_champion:  # Avoid testing the initial champion as target
            guesses = simulate_guesses(target_champion, initial_champion, champions_data)
            champion_guesses[target_champion] = guesses
            if guesses > max_guesses:
                max_guesses = guesses
    
    return max_guesses, champion_guesses




In [3]:
initial_champion = "Lux"
target_champion = "Renata Glasc"

guesses_needed = simulate_guesses(target_champion, initial_champion, champions_data)
print(f"Number of guesses needed for {target_champion}: {guesses_needed}")

guessing: Xayah
guessing: Senna
Number of guesses needed for Renata Glasc: 4


In [42]:
a = simulate_guesses("Pyke", "Lux", champions_data)
a

3

In [50]:
initial_champion = "Zeri"
a = find_max_guesses(initial_champion, champions_data)
a

(7,
 {'Aatrox': 2,
  'Ahri': 2,
  'Akali': 2,
  'Akshan': 2,
  'Alistar': 4,
  'Amumu': 3,
  'Anivia': 3,
  'Annie': 2,
  'Aphelios': 3,
  'Ashe': 2,
  'Aurelion Sol': 2,
  'Aurora': 1,
  'Azir': 2,
  'Bard': 3,
  "Bel'Veth": 1,
  'Blitzcrank': 2,
  'Brand': 3,
  'Braum': 2,
  'Briar': 1,
  'Caitlyn': 2,
  'Camille': 3,
  'Cassiopeia': 3,
  "Cho'Gath": 5,
  'Corki': 3,
  'Darius': 3,
  'Diana': 2,
  'Dr. Mundo': 2,
  'Draven': 2,
  'Ekko': 2,
  'Elise': 2,
  'Evelynn': 2,
  'Ezreal': 1,
  'Fiddlesticks': 2,
  'Fiora': 2,
  'Fizz': 4,
  'Galio': 3,
  'Gangplank': 6,
  'Garen': 2,
  'Gnar': 2,
  'Gragas': 2,
  'Graves': 3,
  'Gwen': 3,
  'Hecarim': 4,
  'Heimerdinger': 4,
  'Hwei': 1,
  'Illaoi': 3,
  'Irelia': 3,
  'Ivern': 3,
  'Janna': 3,
  'Jarvan IV': 3,
  'Jax': 2,
  'Jayce': 2,
  'Jhin': 3,
  'Jinx': 1,
  "K'Sante": 1,
  "Kai'Sa": 3,
  'Kalista': 3,
  'Karma': 2,
  'Karthus': 4,
  'Kassadin': 6,
  'Katarina': 2,
  'Kayle': 2,
  'Kayn': 4,
  'Kennen': 2,
  "Kha'Zix": 4,
  'Kindred'